In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import b3d
import sys

sys.path.append(str(b3d.get_root_path()))

from tests.sama4d.video_to_tracks.keypoint_tracking_task import KeypointTrackingTask
from tests.sama4d.data_curation import get_cheezitbox_scene_loader
import rerun as rr

In [3]:
scene_spec = get_cheezitbox_scene_loader()
task = KeypointTrackingTask(
    scene_spec["feature_track_data_loader"],
    scene_name=scene_spec["scene_name"],
    n_frames=2,
)

In [4]:
rr.init(f'task--{task.name}-003')
rr.connect("127.0.0.1:8812")
task.visualize_task()

In [5]:
task_spec = task.get_task_specification()

In [11]:
from tests.sama4d.video_to_tracks.solvers.particle_system_patch_tracking_solver import (
    AdamPatchTracker_UsingSingleframeParticleSystemTraces,
)

solver = AdamPatchTracker_UsingSingleframeParticleSystemTraces()
solution = solver.solve(task_spec)

In [12]:
metrics = task.score(solution)
metrics

{'mean_distance_error': Array(1.5791383, dtype=float32),
 'n_errors_above_threshold_per_frame': Array([2, 4], dtype=int32)}

In [13]:
task.visualize_solution(solution, metrics)

In [14]:
import rerun.blueprint as rrb

bp = rrb.Blueprint(
    rrb.Horizontal(
        rrb.Spatial3DView(),
        rrb.Vertical(
            rrb.Spatial2DView(
                contents=["groundtruth_solution/keypoints_2d", "solution/keypoints_2D"]
            ),
            rrb.Spatial2DView(),
        ),
    )
)
rr.send_blueprint(bp)

In [15]:
solver.visualize_solver_state(task_spec)

In [ ]:
from tests.sama4d.video_to_tracks.solvers.dense_only_patch_tracking_solver import (
    AdamPatchTracker_UsingDenseOnlyTraces,
)

solver_og = AdamPatchTracker_UsingDenseOnlyTraces()
solution_og = solver_og.solve(task_spec)

In [ ]:
metrics_og = task.score(solution_og)
metrics_og

{'mean_distance_error': Array(1.9814739, dtype=float32),
 'n_errors_above_threshold_per_frame': Array([0, 4, 4, 7, 5, 7, 8, 8, 9, 9], dtype=int32)}

In [ ]:
# from newer version
metrics

{'mean_distance_error': Array(2.096604, dtype=float32),
 'n_errors_above_threshold_per_frame': Array([ 1,  3,  3,  6,  8,  6, 11, 13, 14, 15], dtype=int32)}